In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)




Mounted at /content/drive


In [2]:
import sys
import os
py_file_location = "/content/drive/MyDrive/AI/WORK2/include"
sys.path.append(os.path.abspath(py_file_location))
import helpers
import IPython as IP
import pandas as pd
import numpy as np
#np.seterr(divide='ignore', invalid='ignore')
import matplotlib.pyplot as plt
import librosa
import pickle
import math
import librosa.display
#import warnings
#warnings.filterwarnings('ignore')
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import scipy.io.wavfile
import scipy.io.wavfile

In [3]:
# Load the metadata from the generated CSV
metadata = pd.read_csv('/content/drive/MyDrive/AI/WORK2/data/metadata/test.csv')
metadata.head()

,slice_file_name,class,classID
0,door_ad (1).wav,Knock,0
1,door_ad (3).wav,Knock,0
2,door_ad (4).wav,Knock,0
3,door_ad (5).wav,Knock,0
4,91d28795.wav,Knock,0


In [4]:
# Iterate through all audio files and extract MFCC
features = []
labels = []
frames_max = 0
counter = 0
total_samples = len(metadata)
n_mels = 30
duration = 1 #second
counter2 = 0

frame_length = 16896
hop_length = 8448
overlap_ratio = hop_length/frame_length

In [5]:
for index, row in metadata.iterrows():
    file_path = os.path.join('/content/drive/MyDrive/AI/WORK2/data/wav/test',str(row["slice_file_name"]))
    class_label = row["classID"]

    try:
        # Load audio file
        clip, sr = librosa.load(file_path, sr=16000, mono=True, dtype=np.float32)
        clip_length = len(clip)
        file_nums = math.floor(clip_length / hop_length) - 1

        for i in range(file_nums):
            y = clip[(i * hop_length):(i * hop_length + frame_length)]
            # Normalize audio data between -1 and 1
            #normalized_y = y
           # y = librosa.util.normalize(y)  # y=y/max(abs(y))

            # Compute Mel
            mel = librosa.feature.melspectrogram(y=y, sr=16000, n_fft=1024, hop_length=512,
                                                 n_mels=n_mels, center=False)
            mel = mel/mel.max()

            # Convert sound intensity to log amplitude:
            mel_db = librosa.power_to_db(mel, top_db=80)

            # Normalize MFCC between -1 and 1
            normalized_mel = mel_db
            # normalized_mfcc = librosa.util.normalize(mfcc)   #my_mfcc = mfcc/(np.max(abs(mfcc)))

            mels = normalized_mel

            # Extract MFCCs (do not add padding)
            # mfccs = helpers.get_mfcc(file_path, 0, n_mfcc)

            # Save current frame count
            num_frames = mels.shape[1]

            # Add row (feature / label)
            features.append(mels)
            labels.append(class_label)
            # Update frames maximum
            if (num_frames > frames_max):
                frames_max = num_frames
                print(f"num_frames = {num_frames}")
                print(f"file name : {file_path}")

            # Notify update every N files
            if (counter == 50):
                print("Status: {}/{}".format(index + 1, total_samples))
                counter = 0

            original_name = str(row[0])
            new_name = str(counter2)+'-'+original_name
            split_path = os.path.join('/content/drive/MyDrive/AI/WORK2/data/wav', 'test_split', new_name)
            scipy.io.wavfile.write(split_path, sr, y)

            counter += 1
            counter2+= 1
            if  math.isnan(mel_db[29,31]):
                print(file_path)
    except Exception as e:
           print("Error parsing wavefile: ", e)
   
    

num_frames = 32
file name : /content/drive/MyDrive/AI/WORK2/data/wav/test/door_ad (1).wav
Status: 15/135
Status: 32/135
Status: 41/135
Status: 50/135
Status: 61/135
Status: 63/135
Status: 67/135
Status: 72/135
Status: 74/135
Status: 76/135
Status: 80/135
Status: 83/135
Status: 93/135
Status: 98/135
Status: 98/135
Status: 98/135
Status: 99/135
Status: 100/135
Status: 100/135
Status: 102/135
Status: 102/135
Status: 102/135
Status: 103/135
Status: 103/135
Status: 103/135
Status: 104/135
Status: 104/135
Status: 104/135
Status: 105/135
Status: 106/135
Status: 106/135
Status: 106/135
Status: 114/135
Status: 122/135
Status: 133/135


In [6]:
print(f"frame_max = {frames_max}")
print("Finished: {}/{}".format(index, total_samples))
print(labels)
# Add padding to features with less than frames than frames_max
padded_features = helpers.add_padding(features, frames_max)


frame_max = 32
Finished: 134/135
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 

In [7]:


# Verify shapes
print("Raw features length: {}".format(len(features)))
print("Padded features length: {}".format(len(padded_features)))
print("Feature labels length: {}".format(len(labels)))


# Convert features (X) and labels (y) to Numpy arrays
X = np.array(padded_features)
y = np.array(labels)
print(y)
# Optionally save the features to disk
np.save("/content/drive/MyDrive/AI/WORK2/data/X-mel-test", X)
np.save("/content/drive/MyDrive/AI/WORK2/data/y-mel-test", y)


Raw features length: 1766
Padded features length: 1766
Feature labels length: 1766
[0 0 0 ... 3 3 3]


In [8]:

le = LabelEncoder()
y_test_encoded = to_categorical(le.fit_transform(y))
y_encoded = pd.DataFrame(y_test_encoded)
print(y_encoded)

        0    1    2    3
0     1.0  0.0  0.0  0.0
1     1.0  0.0  0.0  0.0
2     1.0  0.0  0.0  0.0
3     1.0  0.0  0.0  0.0
4     1.0  0.0  0.0  0.0
...   ...  ...  ...  ...
1761  0.0  0.0  0.0  1.0
1762  0.0  0.0  0.0  1.0
1763  0.0  0.0  0.0  1.0
1764  0.0  0.0  0.0  1.0
1765  0.0  0.0  0.0  1.0

[1766 rows x 4 columns]


In [9]:
y_encoded.to_csv('/content/drive/MyDrive/AI/WORK2/data/y_test_mel.csv', index=False, index_label=False, header=False)

X_reshape = X.reshape(len(padded_features), 30*32)
X = pd.DataFrame(X_reshape)
X.to_csv('/content/drive/MyDrive/AI/WORK2/data/X_test_mel.csv', index=False, index_label=False, header=False)
with open('parameter-test-mel.pickle', 'wb') as handle:
    pickle.dump([n_mels, frames_max], handle, protocol=pickle.HIGHEST_PROTOCOL)
print("End program")

End program
